# Importación de librerías

In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import Dense

# Carga de datos

In [112]:
headers = pd.read_csv('../Data/handles-data.csv')
tweets = pd.read_csv('../Data/tweets-data.csv')
pd.options.display.float_format = '{:.2f}'.format

/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Target Clasificación

In [113]:
tweets["nretweets"].isnull().sum()

0

In [114]:
tweets["nretweets"].quantile([0.7,0.8,0.9,0.95,1])

0.70       49.00
0.80      104.00
0.90      311.00
0.95      871.00
1.00   541964.00
Name: nretweets, dtype: float64

In [115]:
tweets["categorias_2"] = pd.qcut(tweets["nretweets"], [0.7,0.8,0.9,0.95,1])#, labels=["very_unpopular", "unpopular","popular", "very_popular"])


In [116]:
tweets["categorias_2"]

0           (104.0, 311.0]
1           (311.0, 871.0]
2           (311.0, 871.0]
3           (311.0, 871.0]
4        (871.0, 541964.0]
               ...        
40576       (104.0, 311.0]
40577                  NaN
40578      (48.999, 104.0]
40579       (104.0, 311.0]
40580       (311.0, 871.0]
Name: categorias_2, Length: 40581, dtype: category
Categories (4, interval[float64, right]): [(48.999, 104.0] < (104.0, 311.0] < (311.0, 871.0] < (871.0, 541964.0]]

## Pipeline

In [117]:
def pipeline_qcut_label(dataset):
    dataset["categorias"] = pd.qcut(dataset["nretweets"], [0.7,0.8,0.9,0.95,1], labels=["very_unpopular", "unpopular","popular", "very_popular"])
    dataset["categorias"][dataset["categorias"].isnull()] = "very_unpopular"
    df = pd.get_dummies(tweets, columns=["categorias"])
    return df[["categorias_very_unpopular","categorias_unpopular","categorias_popular","categorias_very_popular"]]

In [118]:
# tweets_label_categorical = pipeline_qcut_label(tweets)

In [119]:
# tweets_label_categorical

Establezco todos los tweets como poco populares de entrada, así si exiten nulos estos estarán como "poco populares". De esta forma, aquellos tweets que están por debajo o igual al percentil 80 serán como poco populares.

In [120]:
tweets["categorias"] = pd.qcut(tweets["nretweets"], [0.7,0.8,0.90,0.95,1], labels=["very_unpopular", "unpopular","popular", "very_popular"])

In [121]:
tweets["categorias"][tweets["categorias"].isnull()] = "very_unpopular"

In [122]:
tweets["categorias"].head(50)

0          unpopular
1            popular
2            popular
3            popular
4       very_popular
5            popular
6       very_popular
7     very_unpopular
8       very_popular
9          unpopular
10         unpopular
11           popular
12      very_popular
13      very_popular
14      very_popular
15         unpopular
16      very_popular
17      very_popular
18         unpopular
19      very_popular
20      very_popular
21      very_popular
22      very_popular
23      very_popular
24      very_popular
25      very_popular
26      very_popular
27      very_popular
28      very_popular
29      very_popular
30      very_popular
31      very_popular
32      very_popular
33    very_unpopular
34    very_unpopular
35         unpopular
36    very_unpopular
37    very_unpopular
38    very_unpopular
39    very_unpopular
40    very_unpopular
41    very_unpopular
42    very_unpopular
43    very_unpopular
44    very_unpopular
45    very_unpopular
46    very_unpopular
47    very_un

In [145]:
tweets.groupby('categorias').size()

categorias
very_unpopular    32523
unpopular          4006
popular            2023
very_popular       2029
dtype: int64

In [123]:
tweets[tweets["nretweets"]>871]["nretweets"].count()

2029

In [124]:
tweets[tweets["categorias"]=="very_popular"]["categorias"].count()

2029

In [125]:
tweets[tweets["nretweets"]<=104]["nretweets"].count()

32523

In [126]:
tweets[tweets["categorias"]=="very_unpopular"]["categorias"].count()

32523

In [127]:
tweets[tweets["categorias"].isnull()]["categorias"].count()

0

In [128]:
tweets_categorical = pd.get_dummies(tweets, columns=["categorias"])

In [129]:
tweets_categorical.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,reply_to,retweet_date,translate,trans_src,trans_dest,categorias_2,categorias_very_unpopular,categorias_unpopular,categorias_popular,categorias_very_popular
0,1425590913959612419,1425590913959612419,1628722334000.00,2021-08-12 00:52:14,200,NaN,RT @girlsalliance: We're so proud of the four ...,en,[],[],...,[],2021-08-12 00:46:58 CEST,NaN,NaN,NaN,"(104.0, 311.0]",0,1,0,0
1,1427736867739299841,1427736867739299841,1629233969000.00,2021-08-17 22:59:29,200,NaN,Some casual suggestions to 😏SLIDE😏 into when u...,en,"['shoesdaytuesday', 'afterskewlslide']",[],...,[],NaN,NaN,NaN,NaN,"(311.0, 871.0]",0,0,1,0
2,1427667300488937476,1427667300488937476,1629217383000.00,2021-08-17 18:23:03,200,NaN,RT @ValaAfshar: You are not your job.,en,[],[],...,[],2021-08-17 17:05:53 CEST,NaN,NaN,NaN,"(311.0, 871.0]",0,0,1,0
3,1427667012105371652,1427667012105371652,1629217315000.00,2021-08-17 18:21:55,200,NaN,What have we become 😔😂 Toddler Cites Freedom ...,en,[],[],...,[],NaN,NaN,NaN,NaN,"(311.0, 871.0]",0,0,1,0
4,1427497703596990467,1427497703596990467,1629176948000.00,2021-08-17 07:09:08,200,NaN,The tech giants that refuse to massively addre...,en,[],[],...,[],NaN,NaN,NaN,NaN,"(871.0, 541964.0]",0,0,0,1


In [130]:
tweets_categorical["categorias_very_unpopular"]

0        0
1        0
2        0
3        0
4        0
        ..
40576    0
40577    1
40578    1
40579    0
40580    0
Name: categorias_very_unpopular, Length: 40581, dtype: uint8

In [131]:
tweets_categorical[tweets_categorical["categorias_very_unpopular"]<0]["categorias_very_unpopular"].count()

0

In [132]:
print(tweets_categorical[tweets_categorical["categorias_very_unpopular"].isnull()]["categorias_very_unpopular"].count())
print(tweets_categorical[tweets_categorical["categorias_unpopular"].isnull()]["categorias_very_unpopular"].count())
print(tweets_categorical[tweets_categorical["categorias_popular"].isnull()]["categorias_popular"].count())
print(tweets_categorical[tweets_categorical["categorias_very_popular"].isnull()]["categorias_very_popular"].count())

0
0
0
0


id  conversation_id  \
categorias_very_unpopular categorias_unpopular                           
0                         0                      4052             4052   
                          1                      4006             4006   
1                         0                     32523            32523   

                                                created_at   date  timezone  \
categorias_very_unpopular categorias_unpopular                                
0                         0                           4052   4052      4052   
                          1                           4006   4006      4006   
1                         0                          32523  32523     32523   

                                                place  tweet  language  \
categorias_very_unpopular categorias_unpopular                           
0                         0                         0   4052      4052   
                          1                         2   4006      4006   
1                         0                        32  32523     32523   

                                                hashtags  cashtags  ...  \
categorias_very_unpopular categorias_unpopular                      ...   
0                         0                         4052      4052  ...   
                          1                         4006      4006  ...   
1                         0                        32523     32523  ...   

                                                user_rt  retweet_id  reply_to  \
categorias_very_unpopular categorias_unpopular                                  
0                         0                        1043        1043      4052   
                          1                         803         803      4006   
1                         0                        4301        4301     32523   

                                                retweet_date  translate  \
categorias_very_unpopular categorias_unpopular                            
0                         0                             1043          0   
                          1                              803          0   
1                         0                             4301          0   

                                                trans_src  trans_dest  \
categorias_very_unpopular categorias_unpopular                          
0                         0                             0           0   
                          1                             0           0   
1                         0                             0           0   

                                                categorias_2  \
categorias_very_unpopular categorias_unpopular                 
0                         0                             4052   
                          1                             4006   
1                         0                             4225   

                                                categorias_popular  \
categorias_very_unpopular categorias_unpopular                       
0                         0                                   4052   
                          1                                   4006   
1                         0                                  32523   

                                                categorias_very_popular  
categorias_very_unpopular categorias_unpopular                           
0                         0                                        4052  
                          1                                        4006  
1                         0                                       32523  

[3 rows x 41 columns]